### Notebook for the inference of cell-cell communications on COPD-IAV data using `LIANA+`

- **Developed by**: Carlos Talavera-López
- **Würzburg Institute for Systems Immunology, Faculty of Medicine, Julius-Maximilian-Universität Würzburg**
- **Created**: 231109
- **Latest version**: 231109

### Import required modules

In [ ]:
import anndata
import numpy as np
import liana as li
import pandas as pd
import scanpy as sc

from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

### Set up working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

### Read in data

In [ ]:
adata_all = sc.read_h5ad('../../../data/Marburg_cell_states_locked_scANVI_ctl230901.raw.h5ad') 
adata_all

### Check `LIANA+` available methods

In [ ]:
li.mt.show_methods()

### Run `cellphoneDB` for trial

In [ ]:
cellphonedb(adata_all, groupby = 'cell_states', 
            expr_prop = 0.1, 
            resource_name = 'consensus', 
            verbose = True, 
            key_added = 'cpdb_res',
            use_raw = False)
adata_all.uns['cpdb_res'].head()

In [ ]:
adata_all.obs['cell_states'].cat.categories

### Visualise `cellphoneDB` interactions using a dotplot

In [ ]:
li.pl.dotplot(adata = adata_all,
              colour = 'lr_means',
              size = 'cellphone_pvals',
              inverse_size = True, 
              source_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
              target_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
              figure_size = (25, 50),
              filterby = 'cellphone_pvals',
              filter_lambda = lambda x: x <= 0.05,
              uns_key = 'cpdb_res',
              cmap = 'magma'
             )

In [ ]:
my_plot = li.pl.tileplot(adata = adata_all,
                         # NOTE: fill & label need to exist for both
                         # ligand_ and receptor_ columns
                         fill = 'means',
                         label = 'props',
                         label_fun = lambda x: f'{x:.2f}',
                         top_n = 10,
                         orderby = 'cellphone_pvals',
                         orderby_ascending = True,
                         figure_size = (20, 10),
                         source_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
                         target_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
                         uns_key = 'cpdb_res'
                         )
my_plot

### Run `cellchat` for trial

In [ ]:
cellchat(adata_all, groupby = 'cell_states', 
            expr_prop = 0.1, 
            resource_name = 'consensus', 
            verbose = True, 
            key_added = 'ccdb_res',
            use_raw = False)
adata_all.uns['ccdb_res'].head()

In [ ]:
li.pl.dotplot(adata = adata_all,
              colour = 'lr_probs',
              size = 'cellchat_pvals',
              inverse_size = True, 
              source_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
              target_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
              figure_size = (25, 50),
              filterby = 'cellchat_pvals',
              filter_lambda = lambda x: x <= 0.05,
              uns_key = 'ccdb_res',
              cmap = 'magma'
             )

In [ ]:
my_plot = li.pl.tileplot(adata = adata_all,
                         # NOTE: fill & label need to exist for both
                         # ligand_ and receptor_ columns
                         fill = 'cellchat_pvals',
                         label = 'lr_probs',
                         label_fun = lambda x: f'{x:.2f}',
                         top_n = 10,
                         orderby = 'cellchat_pvals',
                         orderby_ascending = True,
                         figure_size = (20, 10),
                         source_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
                         target_labels = ['SERPINE1+Basal', 'SERPINE2+Basal', 'iavAPC_Epi', 'MHCII+Club', 'TNC+Basal'],
                         uns_key = 'ccdb_res'
                         )
my_plot

### Run aggregated estimate 

In [ ]:
li.mt.rank_aggregate(adata_all, groupby = 'cell_states', expr_prop = 0.1, verbose = True, use_raw = False)
adata_all.uns['liana_res'].head()

In [ ]:
rank_aggregate.describe()